In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from multiprocessing import Pool

import numpy as np
import pandas as pd

import kraft

In [ ]:
SETTING = kraft.json.read("setting.json")

In [ ]:
directory_path = "{}/gene_set/".format(SETTING["directory_path"])

kraft.path.path(directory_path)

In [ ]:
gene_x_sample = pd.read_csv(
    "{}/../rna/gene_x_cell_line.tsv".format(directory_path), sep="\t", index_col=0
)

gene_x_sample = gene_x_sample.loc[
    gene_x_sample.index & kraft.name_biology.select_genes(), :
]

In [ ]:
gene_set_to_genes = kraft.gmt.read_many(
    tuple(
        "data/gene_set/{}".format(file_name)
        for file_name in (
            "h.all.v7.1.symbols.gmt",
            "c1.all.v7.1.symbols.gmt",
            "c2.all.v7.1.symbols.gmt",
            "c3.all.v7.1.symbols.gmt",
            "c5.all.v7.1.symbols.gmt",
            "c6.all.v7.1.symbols.gmt",
            "c7.all.v7.1.symbols.gmt",
            "ipa.gmt",
        )
    )
)

In [ ]:
gene_set_x_sample = kraft.sea.score_samples_and_sets(
    gene_x_sample, gene_set_to_genes, n_job=8, directory_path=directory_path
)

gene_set_x_sample